---
title: Implementation of 3SEB with high resolution imagery
subject: Tutorial
subtitle: Notebook to run different 3SEB variants using UAV imagery
authors:
  - name: Vicente Burchard-Levine
    affiliations:
    - SpecLab-CSIC
    orcid: 0000-0003-0222-8706
    email: vicente.burchard@csic.es
  - name: Hector Nieto
    affiliations:
    - ICA-CSIC
    orcid: 0000-0003-4250-6424
    email: hector.nieto@ica.csic.es
---


# Summary 

Interactive jupyter notebook showing the implementation of 3SEB with high resolution imagery obtained from uncrewed aerial imagery (UAVs). 

This notebook will go through the pre-processing of of 3SEB inputs and discuss different versions of 3SEB, including:

- Separating leaf and soil temperatures using contextual methods
- Pre-processing inputs and parameters to force 3SEB
- Running 3SEB variants (3SEB-PT, 3SEB-2T and 3SEB-3T)
- Evaluating flux and temperature outputs against measurements 


# Instructions
Read carefully all the text and follow the instructions.

:::{hint} 

Once each section is read, run the jupyter code cell underneath (marked as `[]`) by clicking the icon `Run`, or pressing the keys SHIFT+ENTER of your keyboard.


:::

To start, please run the following cell to import all the packages required for this notebook. Once you run the cell below, an acknowledgement message, stating all libraries were correctly imported, should be printed on screen.

# Import Libraries

In [ ]:
%matplotlib widget
import numpy as np
from osgeo import gdal
from pathlib import Path
import pandas as pd
import geopandas as gpd
from pyTSEB import TSEB
from pyTSEB import meteo_utils as met
from pyTSEB import net_radiation as rad
from pyTSEB import resistances as res
from pyTSEB import energy_combination_ET as EC
from py3seb import  py3seb 
import matplotlib.pyplot as plt
from osgeo import gdal
import pyDMS.pyDMSUtils as gu
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

print('libraries imported correctly!')

# The leaf and soil temperatures in 3SEB
In 3SEB, the evapotranspiration (ET) and energy fluxes from overstory vegetation, understory vegetation and soil components are solved for each component layer. For adequate flux partitioning accross landscape sources, we need estimates of overstory canopy ($T_C$), understory canopy ($T_{C, sub}$)  and soil ($T_S$) temperatures to retrieve sensible heat fluxes from the different sources. In the orginal formulation of 3SEB, the resistance framework is first treated as a parallel (i.e., uncoupled) tree-substrate system to obtain tree canopy sensible heat flux ($H_C$) and substrate (understory vegetation+soil) ($H_{sub}$) using the heat transport equations:

:::{math}
:label:ec-hc-hsub
H_C &= \rho C_p \frac{T_C - T_{a}} {R_A}\\
H_{sub} &= \rho C_p \frac{T_{sub} - T_{a}} {R_A+R_{sub}}
:::


Subsequently, the substrate fluxes and temperatures are further separated incorporating a series (i.e. coupled) approach:

:::{math}
:label:ec-hcsub-hs
H_{C,sub} &= \rho C_p \frac{T_{C,sub} - T_{ac}} {R_X}\\
H_s &= \rho C_p \frac{T_S - T_{ac}} {R_S}
:::


Since $T_C$, $T_{C,sub}$ and $T_S$ are unknown apriori, the original 3SEB formulation (3SEB-PT) implements a Priestley-Taylor (PT) formulation, as in [Norman et al. (1995)](https://doi.org/10.1016/0168-1923(95)02265-Y), to compute a first estimate of the canopy LE and H for both overstory and understory. 

For more information on the 3SEB-PT resistance scheme, refer to [Burchard-Levine et al., 2022](https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.16002)

However, the use of very high spatial TIR imagery can be especially relevant in row crops with wide spacing and clumped vegetation such as almond orchards where we can directly retrieve these different component temperatures and avoid the initial assumptions of potential transpiration made with the PT formulation. However, in the case of tree crops that have cover crops over the rows, it becomes more difficult to directly separate the cover crop temperature from the soil component. As such, in this exercise, we will consider different 3SEB versions that assume that the temperature retrived over the cover crop refers to canopy temperature $T_{C,sub}$ and another that assumes that is the combined substrate (vegetation+soil) temperature ($T_{sub}$). 

# 3SEB versions

In this notebook, we will implement three different 3SEB versions:

- **3SEB-PT**: implements a double Priestley-Taylor transpiration initialization for each vegetation canopy (i.e. tree and grass)
- **3SEB-2T**: acquires tree and substrate temperature from high resolution imagery and implements a PT initalization in the substrate
- **3SEB-3T**: acquires tree , cover crop and soil temperatures directly from high resolution imagery

Also for comparison purposes, we will also run TSEB-PT. 

:::{warning} 


3SEB-2T and 3SEB-3T are very new developments and have not yet been extensively tested. It has not been published yet and may need further refinements.

This is work in progress so we are happy to receive any feedback!!
:::

:::{important}
Despite that UAV technology can provide very high spatial resolution information we have to be aware that TSEB/3SEB, as most ot the resistance-based energy balance model, must be applied at an adequate spatial domain to comply to the assumptions and the physical formulations inherent in it. Therefore **it is not possible** to run TSEB/3SEB at leaf-level scale (i.e. 10cm), as the formulations of radiative transfer and turbulent exchanche that it relies upon, require to run TSEB/3SEB at a scale in the order of meters.
:::



# Case study and dataset
We will implement 3SEB over an experimental Almond orchard located in California, USA.

:::{figure} ./input/figures/WES_experimental_design_layout.png
:alt: WES experimental design
:name: WES-Design
Experimental design of Almond orchards with different irrigation treatments and cover crop (CC) presence (surface irrgation+CC, surface irrigation+NoCC, subsurface irrigation+CC, subsurface irrigation+noCC)
:::

:::{note}
## To reduce computational needs, we will work only on the surface cover crop treatment (Conv + CC or Surface_CC). 
:::


# Generating 3SEB inputs: *separating temperature components*

Since the UAV imagery have a very high spatioal resolution (<10cm), we can retrive effective leaf/canopy and soil temperatures directly from the imagery. 

To effectively separate the different temperature components, we will use both NDVI and canopy height models (CHM) as constraints. The later especially useful to separate tree and cover crop temperatures.

:::{note}
This is just an example of one simple method that can be used to retrieve the different component temperatures. Other methods can also be applied.
:::

## Open and visualize imagery 

In [ ]:
# workdir 
datadir = Path('./dataset')
uav_dir = datadir / 'uav_imagery'

# ndvi
ndvi_file = uav_dir / f'ndvi_surface_CC_10cm.tif'
ndvi_fid = gdal.Open(str(ndvi_file))
ndvi_ar = ndvi_fid.GetRasterBand(1).ReadAsArray()

# canopy height
ch_file = uav_dir / f'chm_surface_CC_10cm.tif'
ch_fid = gdal.Open(str(ch_file))
ch_ar = ch_fid.GetRasterBand(1).ReadAsArray()
#ch_ar[ch_ar<0] = np.nan
ch_ar[ch_ar>20] = np.nan

# land surface temperature
lst_file = uav_dir / f'lst_surface_CC_10cm.tif'
lst_fid = gdal.Open(str(lst_file))
lst_ar = lst_fid.GetRasterBand(1).ReadAsArray()

# use LST as template for raster info
## Get raster size
cols = lst_fid.RasterXSize
rows = lst_fid.RasterYSize
## geotransform
gt = lst_fid.GetGeoTransform()
minx, maxy = gt[0], gt[3]
maxx = minx + gt[1]*cols
miny = maxy + gt[5]*rows
## extent
te = [minx, maxx, miny, maxy]
## spatial projection
proj = lst_fid.GetProjection()

# plot UAV imagery inputs to visualize
fig, axes = plt.subplots(2,3, figsize = (14,10), constrained_layout=True)
ax1 = axes[0,0]
im1 = ax1.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te)
ax1.set_title('LST')

ax2 = axes[0,1]
im2 = ax2.imshow(ndvi_ar, cmap='YlGn', vmin=0.3, vmax=0.8, extent = te)
ax2.set_title('NDVI')

ax3 = axes[0,2]
im3 = ax3.imshow(ch_ar, cmap='viridis', vmin=0., vmax=5, extent = te)
ax3.set_title('Canopy Height (CH)')


# zoom in
ax4 = axes[1,0]
im4 = ax4.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te)
ax4.set_xlim(655290, 655320)
ax4.set_ylim(4156800, 4156830)
cb = plt.colorbar(im4, ax=ax4, orientation='horizontal')
cb.set_label('LST (K)', fontsize=14)

ax5 = axes[1,1]
im5 = ax5.imshow(ndvi_ar, cmap='YlGn', vmin=0.3, vmax=0.8, extent = te)
ax5.set_xlim(655290, 655320)
ax5.set_ylim(4156800, 4156830)
cb = plt.colorbar(im5, ax=ax5, orientation='horizontal')
cb.set_label('NDVI (-)', fontsize=14)

ax6 = axes[1,2]
im6 = ax6.imshow(ch_ar, cmap='viridis', vmin=0., vmax=5, extent = te)
ax6.set_xlim(655290, 655320)
ax6.set_ylim(4156800, 4156830)
cb = plt.colorbar(im6, ax=ax6, orientation='horizontal')
cb.set_label('CH (m)', fontsize=14)
plt.show()

## Obtaining component tempeatures from contextual method
We will obtain $T_C$, $T_{C,sub}$ and $T_S$ by searching for pure vegetation and soil pixels in a contextual spatial domain . That is, in a coarse resolution spatial grid, consistent with the TSEB physical assumptions, we will assign for each of its cells the canopy and soil temperatures corresponding to the average temperature for the original LST resoltuion pixels that are considered, respectively, bare soil, cover crop vegetation and tree crop vegetation. 

:::{note}
In this case we assume that TSEB will be run at 8 m spatial resolution, as it is spatial domain large enough to include the canopy/interrow system in order to be consistent with the radiative transfer modelling in the rows. 
:::

In this example, we simply consider:
- tree crops as pixels with NDVI > 0.8 & canopy heights > 1.5m
- cover crops as pixels with NDVI > 0.8 & canopy heights < 0.4m
- soil as pixels with NDVI < 0.4 & canopy heights < 0.4

We will create masks for each of these conditions to isolate the component temperatures of each landscape source (i.e., tree crop, cover crop and soil)


:::{warning}
Note that these pure vegetation and soil NDVI values, along with crop height, might change due to different spectral cameras and different crop/soil types.
:::

:::{note}
Other more sophisticated classifcation algorithms (e.g. LULC classificationn with RF) can be applied to better separate the different landscape components. This is just a simple illustrative example.
:::

In [ ]:
# RGB file 
rgb_file = uav_dir / f'rgb_surface_CC_10cm.tif'
rgb_fid = gdal.Open(str(rgb_file))
blue_ar = rgb_fid.GetRasterBand(1).ReadAsArray()
green_ar = rgb_fid.GetRasterBand(2).ReadAsArray()
red_ar = rgb_fid.GetRasterBand(2).ReadAsArray()
# stack arrays
rgb_image = np.dstack((red_ar, green_ar, blue_ar))
# make image brighter for visualization
rgb_image = np.clip(rgb_image * 1.5, 0, 255) / 255

# set up masks to isolate different 
# overstory mask 
mask_ov = np.logical_and(ndvi_ar > 0.8, ch_ar > 1.5)

# understory mask 
mask_un = np.logical_and(ndvi_ar > 0.8, ch_ar < 0.4)

# soil mask 
mask_soil = np.logical_and(ndvi_ar < 0.4, ch_ar < 0.4)

# substrate mask 
mask_sub = np.logical_or(mask_un, mask_soil)

# Tc
Tc_ar = lst_ar.copy()
## all non-tree pixels as nan
Tc_ar[~mask_ov] = np.nan

# Tc_sub
Tc_sub_ar = lst_ar.copy()
## all non-grass pixels as nan
Tc_sub_ar[~mask_un] = np.nan

#Ts
Ts_ar = lst_ar.copy()
## all non-soil pixels as nan
Ts_ar[~mask_soil] = np.nan

#Tsub 
Tsub_ar = lst_ar.copy()
## all non-substrate pixels as nan
Tsub_ar[~mask_un] = np.nan

## create classification array
class_ar = lst_ar.copy()
class_ar[:] = np.nan
class_ar[mask_ov] = 1
class_ar[mask_un] = 2
class_ar[mask_soil] = 3

# Define discrete colors for each category (1=tree, 2=grass, 3=soil)
cmap = ListedColormap(['darkgreen', 'yellowgreen', 'saddlebrown'])

# Define a normalization so the values map to 0, 1, 2 in the colormap
bounds = [0.5, 1.5, 2.5, 3.5]
norm = plt.matplotlib.colors.BoundaryNorm(bounds, cmap.N)


fig, axes = plt.subplots(2,2, figsize = (12,10), constrained_layout=True)
ax1 = axes[0,0]
img = ax1.imshow(class_ar, cmap=cmap, norm=norm, extent=te)

# Create legend manually
legend_labels = {
    1: 'Tree Crop',
    2: 'Cover crop',
    3: 'Soil'
}
legend_colors = ['darkgreen', 'yellowgreen', 'saddlebrown']

patches = [mpatches.Patch(color=legend_colors[i], label=legend_labels[i+1]) for i in range(3)]
ax1.legend(handles=patches, loc='upper right')
#ax1.set_xlim(655290, 655320)
#ax1.set_ylim(4156750, 4156780)

ax2 = axes[0,1]
ax2.imshow(class_ar, cmap=cmap, norm=norm, extent = te, alpha=1)
#ax2.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te, alpha=0.8)

ax2.set_xlim(655288, 655320)
ax2.set_ylim(4156800, 4156830)

ax3 = axes[1,0]
im3 = ax3.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te)

ax3.set_xlim(655288, 655320)
ax3.set_ylim(4156800, 4156830)
#cb = plt.colorbar(im3, ax=ax3, orientation='vertical', shrink = 0.65)
#cb.set_label('LST (K)', fontsize=14)
ax4 = axes[1,1]
im4 = ax4.imshow(rgb_image, extent = te)
ax4.set_xlim(655288, 655320)
ax4.set_ylim(4156800, 4156830)
#ax3.set_xlim(655290, 655320)
#cb.set_label('CH (m)', fontsize=14)
plt.show()

# Save component temperature imagery and fractional covers

We will take advantage of this high resoltion imagery to calcualte the fractional covers of each landscape source (tree crop, cover crop and soil). We will save these as geotiff to then later resampled to 8m as inputs for TSEB/3SEB. 

In [ ]:
# fc (tree canopy fraction)
fc_ar = lst_ar.copy()
fc_ar[mask_ov] = 1
fc_ar[~mask_ov] = 0

# fs (soil fraction)
fs_ar = lst_ar.copy()
fs_ar[mask_soil] = 1
fs_ar[~mask_soil] = 0

# fc_sub (cover crop fraction)
fc_sub_ar = lst_ar.copy()
fc_sub_ar[mask_un] = 1
fc_sub_ar[~mask_un] = 0

# set up filename for outputs (fractional covers)
## tree crop fraction
outfile_fc_mask = uav_dir / f'Fc_surface_CC_10cm.tif'
## cover crop fraction
outfile_fc_sub_mask = uav_dir / f'Fc_sub_surface_CC_10cm.tif'
## soil fraction
outfile_fs_mask = uav_dir / f'Fs_sub_surface_CC_10cm.tif'

# set up filename for outputs (component temperatures)
outfile_Tc = uav_dir / f'Tc_surface_CC_10cm.tif'
outfile_Tc_sub = uav_dir / f'Tc_sub_surface_CC_10cm.tif'
outfile_Ts = uav_dir / f'Ts_surface_CC_10cm.tif'
outfile_Tsub= uav_dir / f'Tsub_surface_CC_10cm.tif'

rows, cols = np.shape(lst_ar)

print('Saving inputs at 10cm...')
# Save output
input_nodata = np.nan
driver = gdal.GetDriverByName('GTiff')
# Tc
ds = driver.Create(str(outfile_Tc), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)
band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(Tc_ar)
band.FlushCache()
ds.FlushCache()
del ds

#Tc_sub
ds = driver.Create(str(outfile_Tc_sub), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)
band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(Tc_sub_ar)
band.FlushCache()
ds.FlushCache()
del ds 

#Ts
ds = driver.Create(str(outfile_Ts), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)
band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(Ts_ar)
band.FlushCache()
ds.FlushCache()
del ds

#Tsub
ds = driver.Create(str(outfile_Tsub), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)
band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(Tsub_ar)
band.FlushCache()
ds.FlushCache()
del ds

#Fc
ds = driver.Create(str(outfile_fc_mask), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)

band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(fc_ar)
band.FlushCache()
ds.FlushCache()
del ds 

#Fs
ds = driver.Create(str(outfile_fs_mask), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)

band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(fs_ar)
band.FlushCache()
ds.FlushCache()
del ds 

#Fc_sub
ds = driver.Create(str(outfile_fc_sub_mask), cols, rows, 1, gdal.GDT_Float32)
ds.SetGeoTransform(gt)
ds.SetProjection(proj)
band = ds.GetRasterBand(1)
band.SetNoDataValue(input_nodata)
band.WriteArray(fc_sub_ar)
band.FlushCache()
ds.FlushCache()
del ds

print('Done!')

:::{note}
You can also visualize these outputs in QGIS
:::

# Obtain average component temperatures over 8m grids 

We now need to obtain the average component temperatures at 8m grids. The next code block visualizes this. 

In [ ]:
# visualize grid layers
vector_dir = datadir / 'vector_data'
grids_file = vector_dir / 'Grids_8m_surface_CC.geojson'
grids_df = gpd.read_file(grids_file, on_invalid="warn")


fig, axes = plt.subplots(1,3, figsize = (14,6), constrained_layout=True)
ax1 = axes[0]
img = ax1.imshow(rgb_image, extent=te)
grids_df.plot(ax=ax1, facecolor='none', edgecolor='white', linewidth=2, alpha=0.9, label='8m grids')

legend_element = mpatches.Patch(facecolor='none', edgecolor='white', label='8m grids')

ax1.legend(handles=[legend_element])

ax1.set_xlim(655288, 655320)
ax1.set_ylim(4156800, 4156830)
ax1.set_title('RGB', fontsize=16)
#ax1.set_xlim(655290, 655320)
#ax1.set_ylim(4156750, 4156780)

# Define discrete colors for each category (1=tree, 2=grass, 3=soil)
cmap = ListedColormap(['darkgreen', 'yellowgreen', 'saddlebrown'])

# Define a normalization so the values map to 0, 1, 2 in the colormap
bounds = [0.5, 1.5, 2.5, 3.5]
norm = plt.matplotlib.colors.BoundaryNorm(bounds, cmap.N)
ax2 = axes[1]
ax2.imshow(class_ar, cmap=cmap, norm=norm, extent = te, alpha=1)
#grids_df.plot(ax=ax2, facecolor='none', edgecolor='orange', linewidth=1)
grids_df.plot(ax=ax2, facecolor='none', edgecolor='k', linewidth=2, alpha=0.9, label='8m grids')

legend_element = mpatches.Patch(facecolor='none', edgecolor='k', label='8m grids')

ax2.legend(handles=[legend_element])
#ax2.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te, alpha=0.8)

ax2.set_xlim(655288, 655320)
ax2.set_ylim(4156800, 4156830)
ax2.set_title('Pure component pixel selection', fontsize=16)

ax3 = axes[2]
im3 = ax3.imshow(lst_ar, cmap='Spectral_r', vmin=300, vmax=320, extent = te)
grids_df.plot(ax=ax3, facecolor='none', edgecolor='k', linewidth=2, alpha=0.9, label='8m grids')

ax3.set_xlim(655288, 655320)
ax3.set_ylim(4156800, 4156830)
cb = plt.colorbar(im3, ax=ax3, orientation='vertical', shrink = 0.65)
cb.set_label('LST (K)', fontsize=14)
ax3.set_title('LST', fontsize=16)
ax3.legend(handles=[legend_element])

plt.show()



## Getting average source values with GDAL

In this case, we will simply calculate the average LST values of each surface source (tree crop, cover and soil) at the 8m scale by using a mask for each source. For example, for tree crops, we mask out all non-tree crop pixels as 'nan' at 10cm scale and then get average values of all non-nan pixels within each 8m grid. 

### save resampled grids to file

In [ ]:
# resample to 8m 
target_res = 8

# set up outputfile name
outfile_fc_8m = uav_dir / f'Fc_surface_CC_{target_res}m.tif'
outfile_fc_sub_8m = uav_dir / f'Fc_sub_surface_CC_{target_res}m.tif'
outfile_fs_8m = uav_dir / f'Fs_surface_CC_{target_res}m.tif'

outfile_Tc_8m = uav_dir / f'Tc_surface_CC_{target_res}m.tif'
outfile_Tc_sub_8m = uav_dir / f'Tc_sub_surface_CC_{target_res}m.tif'
outfile_Ts_8m = uav_dir / f'Ts_surface_CC_{target_res}m.tif'
outfile_Tsub_8m = uav_dir / f'Tsub_surface_CC_{target_res}m.tif'

outfile_lst_8m = uav_dir / f'lst_surface_CC_{target_res}m.tif'

# also chm 
outfile_chm_8m = uav_dir / f'Hc_surface_CC_{target_res}m.tif'
print(f'Resampling inputs to {target_res}m...')

# fractional covers
gdal.Warp(str(outfile_fc_8m), str(outfile_fc_mask), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')
gdal.Warp(str(outfile_fc_sub_8m), str(outfile_fc_sub_mask), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')
gdal.Warp(str(outfile_fs_8m), str(outfile_fs_mask), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')

# component temperatures
gdal.Warp(str(outfile_Tc_8m), str(outfile_Tc), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'min')
gdal.Warp(str(outfile_Tc_sub_8m), str(outfile_Tc_sub), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'min')
gdal.Warp(str(outfile_Ts_8m), str(outfile_Ts), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')
gdal.Warp(str(outfile_Tsub_8m), str(outfile_Tsub), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')

gdal.Warp(str(outfile_lst_8m), str(lst_file), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'average')

# Also canopy height (get maximum in this case as this best describes surface roughness)
gdal.Warp(str(outfile_chm_8m), str(ch_file), format='Gtiff', xRes=target_res, yRes=target_res, resampleAlg= 'max')

print(f'Done!')

# Visualize resampled inputs

We can now open and visualize the temperature inputs we will use to run TSEB/3SEB

In [ ]:
# LST
lst_file = uav_dir / 'lst_surface_CC_8m.tif'
lst_fid = gdal.Open(str(lst_file))
lst_ar = lst_fid.GetRasterBand(1).ReadAsArray()
lst_ar[np.logical_or(lst_ar<275,lst_ar>400)] = np.nan

# Tree crop temperature
Tc_file = uav_dir / 'Tc_surface_CC_8m.tif'
Tc_fid = gdal.Open(str(Tc_file))
Tc_ar = Tc_fid.GetRasterBand(1).ReadAsArray()
Tc_ar[np.logical_or(Tc_ar<275,Tc_ar>400)] = np.nan

# Cover crop temperature
Tcc_file = uav_dir / 'Tc_sub_surface_CC_8m.tif'
Tcc_fid = gdal.Open(str(Tcc_file))
Tcc_ar = Tcc_fid.GetRasterBand(1).ReadAsArray()
Tcc_ar[np.logical_or(Tcc_ar<275,Tcc_ar>400)] = np.nan

# Soil temperature
Ts_file = uav_dir / 'Ts_surface_CC_8m.tif'
Ts_fid = gdal.Open(str(Ts_file))
Ts_ar = Ts_fid.GetRasterBand(1).ReadAsArray()
Ts_ar[np.logical_or(Ts_ar<200,Ts_ar>500)] = np.nan

# Soil temperature
Tsub_file = uav_dir / 'Tsub_surface_CC_8m.tif'
Tsub_fid = gdal.Open(str(Tsub_file))
Tsub_ar = Tsub_fid.GetRasterBand(1).ReadAsArray()
Tsub_ar[np.logical_or(Tsub_ar<200,Tsub_ar>500)] = np.nan



# tree crop fractional cover
Fc_file = uav_dir / 'Fc_surface_CC_8m.tif'
Fc_fid = gdal.Open(str(Fc_file))
Fc_ar = Fc_fid.GetRasterBand(1).ReadAsArray()
Fc_ar[np.isnan(lst_ar)] = np.nan

# soil fractional cover
Fs_file = uav_dir / 'Fs_surface_CC_8m.tif'
Fs_fid = gdal.Open(str(Fs_file))
Fs_ar = Fs_fid.GetRasterBand(1).ReadAsArray()
Fs_ar[np.isnan(lst_ar)] = np.nan


# Cover crop fractional cover
Fcc_file = uav_dir / 'Fc_sub_surface_CC_8m.tif'
Fcc_fid = gdal.Open(str(Fcc_file))
Fcc_ar = Fcc_fid.GetRasterBand(1).ReadAsArray()
Fcc_ar[np.isnan(lst_ar)] = np.nan

# the above Fcc refers to total landscape fraction of cover crop
# we need to adjust this to obtain the fraction of cover crop over the substrate (cover crop + soil)
# because this is the input that is needed in 3SEB
Fcc_ar = Fcc_ar / (Fcc_ar+Fs_ar)

# Tree crop Canopy height model
chm_file = uav_dir / 'Hc_surface_CC_8m.tif'
Hc_fid = gdal.Open(str(chm_file))
Hc_ar = Hc_fid.GetRasterBand(1).ReadAsArray()
Hc_ar[np.logical_or(Hc_ar==0,Hc_ar>10)] = np.nan

# get raster metadata 
 # get extent and info from reference dataset
prj = lst_fid.GetProjection()
ulx, xres, xskew, uly, yskew, yres = lst_fid.GetGeoTransform()
lrx = ulx + (lst_fid.RasterXSize * xres)
lry = uly + (lst_fid.RasterYSize * yres)

te = [minx, lrx, lry, uly]

# plot UAV-based inputs
fig, axes = plt.subplots(2,2, figsize=(12,10))
ax = axes[0,0]
im = ax.imshow(lst_ar, vmin=290, vmax=320, cmap='coolwarm', extent = te)
ax.set_title('LST - Land Surface Temperature')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('LST (K)')  # Add title to colorbar
mean_value = np.nanmean(lst_ar)
ax.text(1, 70, f'mean: {np.round(mean_value,2)}')
ax = axes[0,1]
im = ax.imshow(Tc_ar, vmin=290, vmax=320, cmap='coolwarm',  extent = te)
ax.set_title('$T_c$ - Tree crop temperature')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('$T_c$ (K)')  # Add title to colorbar
mean_value = np.nanmean(Tc_ar)
ax.text(1, 70, f'mean: {np.round(mean_value,2)}')

ax = axes[1,0]
im = ax.imshow(Tcc_ar, vmin=290, vmax=320, cmap='coolwarm',  extent = te)
ax.set_title('$T_{c,sub}$ - Cover crop temperature')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('$T_c$ (K)')  # Add title to colorbar

ax = axes[1,1]
im = ax.imshow(Ts_ar, vmin=290, vmax=320, cmap='coolwarm',  extent = te)
ax.set_title('$T_S$ - Soil temperature')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Ts (K)')  # Add title to colorbar

plt.show()

# Other inputs and parameters

Now that we have the main UAV-based inputs, we will now set up the rest of parameters and inputs to run TSEB/3SEB

## Meterological forcings
obtain meteo/EC data needed to run and evaluate TSEB/3SEB. In this case, we will obtain it from the in-situ EC tower located in the study site.

In [ ]:
# meteo
meteo_file = datadir /'insitu'/ 'WES_CSI_Format.csv'
meteo_ds = pd.read_csv(meteo_file, skiprows = [0,2,3])
# soil data
soil_file =  datadir /'insitu'/ 'WES_Soil.dat'
soil_ds = pd.read_csv(soil_file, sep='\t')
meteo_ds

## Extract Meteo data over overpass time

The UAV imagery was acquired on **2024-04-16** between **10:45-11:15** (standard time) so we take the sampling timestep of **11:00** to extract the meterological forcings. 


In [ ]:
# study site information
lat = 37.5456694
lon = -121.2420336
stdlon = -120

# height of sensor measurements
z_u = 5.5 # height wind speed measurement(m)
z_t = 5.5 # height air temperature measurement(m)


date_mask = meteo_ds['TIMESTAMP'] == '4/16/24 11:00'
date_ts = pd.to_datetime('16/4/24 11:00')

hour = date_ts.hour

doy = date_ts.dayofyear

# sun zenith and azimuth angles
sza, saa = met.calc_sun_angles(np.ones_like(Tc_ar) * lat,
                                   np.ones_like(Tc_ar) * lon,
                                   np.ones_like(Tc_ar) * stdlon,
                                   np.ones_like(Tc_ar) * doy,
                                   np.ones_like(Tc_ar) * hour)
sza[sza > 90] = 90

Ta = meteo_ds['TA_3_1_1'].values[date_mask][0] + 273.15
Ta_ar = np.ones(Tc_ar.shape) * Ta

Ea = meteo_ds['e'].values[date_mask][0] * 10
Ea_ar = np.ones(Tc_ar.shape) * Ea

U = meteo_ds['WS'].values[date_mask][0]
U_ar = np.ones(Tc_ar.shape) * U

P = meteo_ds['PA'].values[date_mask][0] * 10
P_ar = np.ones(Tc_ar.shape) * P

Sdn = meteo_ds['SW_IN'].values[date_mask][0]
Sdn_ar = np.ones(Tc_ar.shape) * Sdn
Sdn_ar[sza > 90] = 0

Sup = meteo_ds['SW_OUT'].values[date_mask][0]

Ldn = meteo_ds['LW_IN'].values[date_mask][0]
Ldn_ar = np.ones(Tc_ar.shape) * Ldn

Lup = meteo_ds['LW_OUT'].values[date_mask][0]

# save energy fluxes to outdict
LE_obs = meteo_ds['LE'].values[date_mask][0]

H_obs = meteo_ds['H'].values[date_mask][0]

Rn_obs = meteo_ds['NETRAD'].values[date_mask][0]

# get IRT T_canopy
T_canopy = meteo_ds['T_CANOPY'].values[date_mask][0] + 273.15

# get soil heat flux data from soil dataset
soil_date_mask = soil_ds['TIMESTAMP'] == '4/16/24 11:00'
G_obs = soil_ds['G'].values[soil_date_mask][0]


# create an output dictionary to store results to then compare with model results
outdict = {'date':[], 'hour':[], 'SWin_obs':[], 'SWout_obs':[],'LWin_obs':[], 'LWout_obs':[], 'Rn_obs':[], 'LE_obs':[], 'H_obs':[], 'G_obs':[], 'IRT_Canopy':[]}
outdict['date'].append(date_ts) # date 
outdict['hour'].append(hour) # hour 
outdict['SWin_obs'].append(Sdn) # shortwave incoming radiation 
outdict['SWout_obs'].append(Sup) # shortwave outgoing radiation 
outdict['LWin_obs'].append(Ldn) # longwave incoming radiation 
outdict['LWout_obs'].append(Lup) # longwave outgoing radiation 
outdict['Rn_obs'].append(Rn_obs) # Net radiation
outdict['LE_obs'].append(LE_obs) # latent heat flux
outdict['H_obs'].append(H_obs) # sensible heat flux
outdict['G_obs'].append(G_obs) # soil heat flux
outdict['IRT_Canopy'].append(T_canopy) # IRT canopy temperature

print('Done!')

## Calculating vegetation biophysical variables

:::{warning}
For now, we are assuming constant LAI for both Almonds and cover crops. This should ideally be estimated using multispectral and/or 3D point cloud data fitting an empirical model or inversting a radiative transfer model (RTM). You can see ./302-Biophysical_Traits_RTM.ipynb for an example on how to estimate biophysical traits using satellite imagery and RTMs, which could also be applied for UAV imagery.
:::

In [ ]:
# for now constant values for LAI
## tree crop
F = 2.5
lai_ar = Fc_ar * F

## cover crop
lai_cc_ar = np.ones_like(Tc_ar) * 0.6

# get 'ecosystem' level biophysical variables (combine tree and cover crops) for TSEB
lai_eco = lai_ar + lai_cc_ar 
Fc_eco = Fc_ar + Fcc_ar
Fc_eco[Fc_eco>1] = 1
F_eco = lai_eco/Fc_eco

# cover crop height
Hc_cc_ar = np.ones_like(Tc_ar) * 0.4

# green fraction
## tree crop
Fg_ar = np.ones_like(Tc_ar) * 0.9 # less than 1 to take into account woody/non-green material
## cover crop
Fg_cc_ar = np.ones_like(Tc_ar) * 1.

# local LAI of cover crop
F_sub = lai_cc_ar/Fcc_ar

# row architecture and width
row_direction = 90

## width of interrow
interrow = 8

# canopy width to height ratio
wc_ratio = 1
## cover crop
wc_sub_ratio = 1

# leaf width
## tree crop
lw = 0.05
## cover crop
lw_cc = 0.01
x_lad = 1

# to take into account row strucure on vegetation clumping
psi = row_direction - saa

# calculate clumping index

## tree crop
Omega0 = TSEB.CI.calc_omega0_Kustas(lai_ar, Fc_ar, x_LAD=x_lad, isLAIeff=True)
Omega = TSEB.CI.calc_omega_rows(lai_ar, Fc_ar, theta=sza,
                                psi=psi, w_c=wc_ratio,
                                x_lad=x_lad)
# effective LAI (tree crop)
lai_eff =  F * Omega

## understory vegetation
Omega0_un = TSEB.CI.calc_omega0_Kustas(lai_cc_ar, Fcc_ar, x_LAD=x_lad, isLAIeff=True)
Omega_un = TSEB.CI.calc_omega_Kustas(Omega0_un, sza, w_C=wc_sub_ratio)

# effective LAI (cover crop)
lai_cc_eff =  F_sub * Omega_un

# ecosystem lai 
Omega0_eco = TSEB.CI.calc_omega0_Kustas(lai_eco, Fc_eco, x_LAD=x_lad, isLAIeff=True)
Omega_eco = TSEB.CI.calc_omega_Kustas(Omega0_eco, sza, w_C=wc_sub_ratio)
lai_eco_eff = F_eco * Omega_eco
print('Done!')

## Ancillary information and parameters

In [ ]:
#==============================================================================
# Canopy and Soil spectra
#==============================================================================

spectraVeg = {'rho_leaf_vis': 0.05, 'tau_leaf_vis': 0.08, 'rho_leaf_nir': 0.26, 'tau_leaf_nir': 0.33}  # from pyTSEB
spectraGrd = {'rsoilv': 0.07, 'rsoiln': 0.28}


#Thermal spectra
e_v_constant=0.99        #Leaf emissivity
e_s_constant=0.95        #Soil emissivity

# viewing angle of sensor
vza = 0

# for now keep constant for both layers
z0_soil=0.01

# TSEB parameters
KN_c = 0.0038  # Kondo & Ishida (1997) coefficient for rough surfaces
KN_b = 0.0120  # Kustas and Norman (1999) after Sauer and Norman (1995)

alpha_pt = 1.26 # alpha parameter in Priestley-Taylor Initialization

# using constant ratio appraoch to estimate G
G_constant = 0.35
calcG = [[1], G_constant]

# use Norman and Kustas 1995 resistance framework
Resistance_flag=[0,{}]


# Radiation partionning

Net Radiation intercepted at the canopy and transmitted to the soil is estimated using the [Campbell & Norman (1998)](https://link.springer.com/book/10.1007/978-1-4612-1626-1) model (see Chapter 15).

In the case of 3SEB, an adapted 3-source version is implemented taking into account the two vegetation layers in addition to a soil layer. See Burchard-Levine et al. 2022 for more details

### 2-Source Net Radiation Modeling
The [Campbell & Norman (1998)](https://link.springer.com/book/10.1007/978-1-4612-1626-1) model is used to simulate shortwave radiation partitioning between vegetation and soil 

In [ ]:
# Emissivity
e_s = np.ones(lst_ar.shape) * e_s_constant
e_v = np.ones(lst_ar.shape) * e_v_constant

difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(Sdn_ar, sza, press=P_ar)
skyl = fvis * difvis + fnir * difnir
Sdn_dir = (1. - skyl) * Sdn_ar
Sdn_dif = skyl * Sdn_ar

# incoming long wave radiation
emisAtm = rad.calc_emiss_atm(Ea_ar, Ta_ar)
Lsky = emisAtm * met.calc_stephan_boltzmann(Ta_ar)

sn_veg, sn_soil = TSEB.rad.calc_Sn_Campbell(lai_eco, sza, Sdn_dir, Sdn_dif, fvis, fnir,
                                                    np.full_like(lai_ar, spectraVeg['rho_leaf_vis']),
                                                    np.full_like(lai_ar, spectraVeg['tau_leaf_vis']),
                                                    np.full_like(lai_ar, spectraVeg['rho_leaf_nir']),
                                                    np.full_like(lai_ar, spectraVeg['tau_leaf_nir']),
                                                    np.full_like(lai_ar, spectraGrd['rsoilv']),
                                                    np.full_like(lai_ar, spectraGrd['rsoiln']),
                                                    x_LAD=x_lad, LAI_eff=lai_eco_eff) 

sn_veg[~np.isfinite(sn_veg)] = 0
sn_soil[~np.isfinite(sn_soil)] = 0

print('Done!')

### 3-Source Net Radiation Modeling
Estimate shortwave radiation transmission using an adapted Campbell 1998 model (adapted for 3 layers). See the Supplementary Information of [Burchard-Levine et al. (2022)](https://doi.org/10.1111/gcb.16002) for more details. 
- sn_ov = net shortwave radiation for overstory (tree crop)
- sn_un = net shortwave radiation for understory (cover crop)
- sn_soil =  net shortwave radiation for soil 


In [ ]:
# estimate radiation transmission using Campbell 1998 model (adapted for 3 layers)
sn_ov, sn_s, sn_un = py3seb.calc_Sn_Campbell(lai_ar,
                                       lai_cc_ar,
                                       sza,
                                       Sdn_dir,
                                       Sdn_dif,
                                       fvis,
                                       fnir,
                                       np.full_like(lai_ar, spectraVeg['rho_leaf_vis']),
                                       np.full_like(lai_ar, spectraVeg['rho_leaf_vis']),
                                       np.full_like(lai_ar, spectraVeg['tau_leaf_vis']),
                                       np.full_like(lai_ar, spectraVeg['tau_leaf_vis']),
                                       np.full_like(lai_ar, spectraVeg['rho_leaf_nir']),
                                       np.full_like(lai_ar, spectraVeg['rho_leaf_nir']),
                                       np.full_like(lai_ar, spectraVeg['tau_leaf_nir']),
                                       np.full_like(lai_ar, spectraVeg['tau_leaf_nir']),
                                       np.full_like(lai_ar, spectraGrd['rsoilv']),
                                       np.full_like(lai_ar, spectraGrd['rsoiln']),
                                       Hc_ar,
                                       0.25 * Hc_ar,
                                       wc_sub_ratio,
                                       Fc_ar,
                                       LAI_eff=lai_eff,
                                       LAI_eff_sub=lai_cc_eff)

sn_ov[~np.isfinite(sn_ov)] = 0
sn_s[~np.isfinite(sn_s)] = 0
sn_un[~np.isfinite(sn_un)] = 0

print('Done!')

# Landscape roughness estimation 

In [ ]:
# main vegetation layer (i.e. Almond orchard)
#[z_0M, d_0] = TSEB.res.calc_roughness(lai_ar, Hc_ar, np.ones(lai_ar.shape) * wc_ratio,
 #                                     np.ones(lai_ar.shape) * TSEB.res.BROADLEAVED_D)
# calculate tree crop roughness parameters taking into account LAIeff using Raupach 1994 model
z_0M_factor, d_0_factor = py3seb.raupach_94(lai_eff)
d_0 = Hc_ar*d_0_factor
z_0M = Hc_ar*z_0M_factor

d_0[d_0 < 0] = 0
z_0M[z_0M < z0_soil] = z0_soil

# understory/secondary vegetation (i.e. Cover crop)
z_0m_un, d_0_un = TSEB.res.calc_roughness(lai_cc_eff,
                                          Hc_cc_ar,
                                           np.full_like(lai_cc_ar, wc_sub_ratio),
                                          np.full_like(lai_cc_ar, TSEB.res.GRASS))
d_0_un[d_0_un < 0] = 0
z_0m_un[z_0m_un < z0_soil] = z0_soil

print('Done!')

# Running TSEB-PT, 3SEB-PT, 3SEB-2T, 3SEB-3T

We will run the different TSEB/3SEB variants in this section.

For each model run, we will save the modelled outputs in a dictionary to then be able to compare models and agaisnt tower measurements

In [ ]:
model_outdict = {}

## TSEB-PT
Here we will the original TSEB with the priestley-taylor initialization for comparison purposes. 

In [ ]:
[flag_PT_all, T_soil, T_veg, T_AC, Ln_soil, Ln_veg, LE_veg, H_veg,
     LE_soil, H_soil, G_mod, R_S, R_X, R_A, u_friction, L, n_iterations] = TSEB.TSEB_PT(lst_ar,
                                                                                        vza,
                                                                                        Ta_ar,
                                                                                        U_ar,
                                                                                        Ea_ar,
                                                                                        P_ar,
                                                                                        sn_veg,
                                                                                        sn_soil,
                                                                                        Ldn,
                                                                                        lai_eco,
                                                                                        Hc_ar,
                                                                                        e_v,
                                                                                        e_s,
                                                                                        z_0M,
                                                                                        d_0,
                                                                                        z_t,
                                                                                        z_t,
                                                                                        leaf_width=lw,
                                                                                        alpha_PT=alpha_pt,
                                                                                        f_c=Fc_eco,
                                                                                        f_g=Fg_ar,
                                                                                        calcG_params=calcG,
                                                                                        resistance_form=Resistance_flag)

# save ouputs in outdict 
LE = LE_veg + LE_soil
H = H_veg + H_soil
Rn = (Ln_veg + sn_veg) + (Ln_soil + sn_soil)

# save outputs to dictionary
model_outdict['LE_TSEB-PT'] = LE
model_outdict['LEc_TSEB-PT'] = LE_veg
model_outdict['H_TSEB-PT'] = H
model_outdict['Rn_TSEB-PT'] = Rn
model_outdict['G_TSEB-PT'] = G_mod
model_outdict['Tc_TSEB-PT'] = T_veg
model_outdict['Ts_TSEB-PT'] = T_soil
model_outdict['Flags_TSEB-PT'] = flag_PT_all

# visualizing outputs 
fig, axes = plt.subplots(2,2, figsize=(10,8))
ax = axes[0,0]
im = ax.imshow(LE, vmin=0, vmax=600, cmap='PuBu', extent = te)
ax.set_title('LE')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('LE (W/$m^2$)')  # Add title to colorbar
le_mean = int(np.round(np.nanmean(LE),0))
ax.text(0.01,0.1, f'mean:\n{le_mean} W/$m^2$', transform=ax.transAxes)
ax = axes[0,1]
im = ax.imshow(H, vmin=0, vmax=600, cmap='OrRd',  extent = te)
ax.set_title('H')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('H (W/$m^2$)')  # Add title to colorbar
h_mean = int(np.round(np.nanmean(H),0))
ax.text(0.01,0.1, f'mean:\n{h_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,0]
im = ax.imshow(Rn, vmin=0, vmax=800, cmap='plasma',  extent = te)
ax.set_title('Rn')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Rn (W/$m^2$)')  # Add title to colorbar
rn_mean = int(np.round(np.nanmean(Rn),0))
ax.text(0.01,0.1, f'mean:\n{rn_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,1]
im = ax.imshow(G_mod, vmin=0, vmax=200, cmap='copper',  extent = te)
ax.set_title('G')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('G (W/$m^2$)')  # Add title to colorbar
g_mean = int(np.round(np.nanmean(G_mod),0))
ax.text(0.01,0.1, f'mean:\n{g_mean} W/$m^2$', transform=ax.transAxes)

plt.show()


# 3SEB-PT
Here we will run 3SEB-PT, which initializes both vegetation canopies with Priestley-Taylor assumption

In [ ]:
[flag_PT_all, T_S, T_C, T_C_sub, T_AC, L_n_sub, L_nC, Ln_C_sub, Ln_S, LE_C, H_C, LE_C_sub, H_C_sub,
 LE_S, H_S, G_mod, R_S, R_sub, R_X, R_A, u_friction, L, n_iterations] = py3seb.ThreeSEB_PT(lst_ar,
                                                                                         vza,
                                                                                         Ta_ar,
                                                                                         U_ar,
                                                                                         Ea_ar,
                                                                                         P_ar,
                                                                                         sn_ov,
                                                                                         sn_s,
                                                                                         sn_un,
                                                                                         Ldn_ar,
                                                                                         lai_ar,
                                                                                         lai_cc_ar,
                                                                                         Hc_ar,
                                                                                         Hc_cc_ar,
                                                                                         e_v,
                                                                                         e_v,#change e_v cover crop
                                                                                         e_s,
                                                                                         z_0M,
                                                                                         z_0m_un,
                                                                                         d_0,
                                                                                         d_0_un,
                                                                                         z_u,
                                                                                         z_t,
                                                                                         leaf_width=lw,
                                                                                         leaf_width_sub=lw_cc,
                                                                                         f_c=Fc_ar,
                                                                                         f_c_sub=Fcc_ar,
                                                                                         f_g=Fg_ar,
                                                                                         f_g_sub=Fg_cc_ar,
                                                                                         calcG_params=calcG,
                                                                                         resistance_form=Resistance_flag)
# save ouputs in outdict 
LE = LE_C + LE_C_sub + LE_S
H = H_C + H_S + H_C_sub

Rn_C = L_nC + sn_ov
Rn_C_sub = Ln_C_sub + sn_un
Rn_S = Ln_S + sn_s
Rn = Rn_C + Rn_C_sub + Rn_S

# save outputs to dictionary
model_outdict['LE_3SEB-PT'] = LE
model_outdict['LEc_3SEB-PT'] = LE_C
model_outdict['LEcc_3SEB-PT'] = LE_C_sub
model_outdict['H_3SEB-PT'] = H
model_outdict['Rn_3SEB-PT'] = Rn
model_outdict['G_3SEB-PT'] = G_mod
model_outdict['Tc_3SEB-PT'] = T_C
model_outdict['Tcc_3SEB-PT'] = T_C_sub
model_outdict['Ts_3SEB-PT'] = T_S
model_outdict['Flags_3SEB-PT'] = flag_PT_all

# visualizing outputs 

fig, axes = plt.subplots(2,2, figsize=(10,8))
ax = axes[0,0]
im = ax.imshow(LE, vmin=0, vmax=600, cmap='PuBu', extent = te)
ax.set_title('LE')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('LE (W/$m^2$)')  # Add title to colorbar
le_mean = int(np.round(np.nanmean(LE),0))
ax.text(0.01,0.1, f'mean:\n{le_mean} W/$m^2$', transform=ax.transAxes)
ax = axes[0,1]
im = ax.imshow(H, vmin=0, vmax=600, cmap='OrRd',  extent = te)
ax.set_title('H')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('H (W/$m^2$)')  # Add title to colorbar
h_mean = int(np.round(np.nanmean(H),0))
ax.text(0.01,0.1, f'mean:\n{h_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,0]
im = ax.imshow(Rn, vmin=0, vmax=800, cmap='plasma',  extent = te)
ax.set_title('Rn')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Rn (W/$m^2$)')  # Add title to colorbar
rn_mean = int(np.round(np.nanmean(Rn),0))
ax.text(0.01,0.1, f'mean:\n{rn_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,1]
im = ax.imshow(G_mod, vmin=0, vmax=200, cmap='copper',  extent = te)
ax.set_title('G')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('G (W/$m^2$)')  # Add title to colorbar
g_mean = int(np.round(np.nanmean(G_mod),0))
ax.text(0.01,0.1, f'mean:\n{g_mean} W/$m^2$', transform=ax.transAxes)


plt.show()

# 3SEB-2T

In [ ]:
#T_sub = ((Fcc_ar*(Tcc_ar)**4)+((1-Fcc_ar)*(Ts_ar)**4))**0.25

[flag_PT_all, T_S, T_C_sub, T_AC, Ln_sub, Ln_C, Ln_C_sub, Ln_S, LE_C, H_C, LE_C_sub, H_C_sub,
LE_S, H_S, G_mod, R_sub, R_X_un, R_X_ov, R_A, u_friction, L, n_iterations] = py3seb.ThreeSEB_2T(Tc_ar,
                                                                             Tsub_ar,
                                                                             vza,
                                                                             Ta_ar,
                                                                             U_ar,
                                                                             Ea_ar,
                                                                             P_ar,
                                                                             sn_ov,
                                                                             sn_s,
                                                                             sn_un,
                                                                             Ldn_ar,
                                                                             lai_ar,
                                                                             lai_cc_ar,
                                                                             Hc_ar,
                                                                             Hc_cc_ar,
                                                                             e_v,
                                                                             e_v,#change e_v cover crop
                                                                             e_s,
                                                                             z_0M,
                                                                             z_0m_un,
                                                                             d_0,
                                                                             d_0_un,
                                                                             z_u,
                                                                             z_t,
                                                                             leaf_width=lw,
                                                                             leaf_width_sub=lw_cc,
                                                                             f_c=Fc_ar,
                                                                             f_c_sub=Fcc_ar,
                                                                             f_g=Fg_ar,
                                                                             f_g_sub=Fg_cc_ar,
                                                                             calcG_params=calcG,
                                                                             resistance_form=Resistance_flag)

# save ouputs in outdict 
LE = LE_C + LE_C_sub + LE_S
H = H_C + H_S + H_C_sub

Rn_C = Ln_C + sn_ov
Rn_C_sub = Ln_C_sub + sn_un
Rn_S = Ln_S + sn_s
Rn = Rn_C + Rn_C_sub + Rn_S
Rn[Rn>1000] = np.nan

# visualizing outputs 
# save outputs to dictionary
model_outdict['LE_3SEB-2T'] = LE
model_outdict['LEc_3SEB-2T'] = LE_C
model_outdict['LEcc_3SEB-2T'] = LE_C_sub
model_outdict['H_3SEB-2T'] = H
model_outdict['Rn_3SEB-2T'] = Rn
model_outdict['G_3SEB-2T'] = G_mod
model_outdict['Tc_3SEB-2T'] = Tc_ar
model_outdict['Tcc_3SEB-2T'] = T_C_sub
model_outdict['Ts_3SEB-2T'] = T_S
model_outdict['Flags_3SEB-2T'] = flag_PT_all

# visualize outputs
fig, axes = plt.subplots(2,2, figsize=(10,8))
ax = axes[0,0]
im = ax.imshow(LE, vmin=0, vmax=600, cmap='PuBu', extent = te)
ax.set_title('LE')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('LE (W/$m^2$)')  # Add title to colorbar
le_mean = int(np.round(np.nanmean(LE),0))
ax.text(0.01,0.1, f'mean:\n{le_mean} W/$m^2$', transform=ax.transAxes)
ax = axes[0,1]
im = ax.imshow(H, vmin=0, vmax=600, cmap='OrRd',  extent = te)
ax.set_title('H')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('H (W/$m^2$)')  # Add title to colorbar
h_mean = int(np.round(np.nanmean(H),0))
ax.text(0.01,0.1, f'mean:\n{h_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,0]
im = ax.imshow(Rn, vmin=0, vmax=800, cmap='plasma',  extent = te)
ax.set_title('Rn')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Rn (W/$m^2$)')  # Add title to colorbar
rn_mean = int(np.round(np.nanmean(Rn),0))
ax.text(0.01,0.1, f'mean:\n{rn_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,1]
im = ax.imshow(G_mod, vmin=0, vmax=200, cmap='copper',  extent = te)
ax.set_title('G')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('G (W/$m^2$)')  # Add title to colorbar
g_mean = int(np.round(np.nanmean(G_mod),0))
ax.text(0.01,0.1, f'mean:\n{g_mean} W/$m^2$', transform=ax.transAxes)


plt.show()

# Running 3SEB-3T

In [ ]:
[flag_PT_all, T_AC, L_nC, Ln_C_sub, Ln_S, LE_C, H_C, LE_C_sub, H_C_sub,
LE_S, H_S, G_mod, R_S, R_x_un,R_x_ov, R_A, u_friction, L, n_iterations] = py3seb.ThreeSEB_3T(Tc_ar,
                                                                                     Tcc_ar,
                                                                                     Ts_ar,
                                                                                     Ta_ar,
                                                                                     U_ar,
                                                                                     Ea_ar,
                                                                                     P_ar,
                                                                                     sn_ov,
                                                                                     sn_s,
                                                                                     sn_un,
                                                                                     Ldn_ar,
                                                                                     lai_ar,
                                                                                     lai_cc_ar,
                                                                                     Hc_ar,
                                                                                     Hc_cc_ar,
                                                                                     e_v,
                                                                                     e_v,#change e_v cover crop
                                                                                     e_s,
                                                                                     z_0M,
                                                                                     z_0m_un,
                                                                                     d_0,
                                                                                     d_0_un,
                                                                                     z_u,
                                                                                     z_t,
                                                                                     leaf_width=lw,
                                                                                     leaf_width_sub=lw_cc,
                                                                                     f_c=Fc_ar,
                                                                                     f_c_sub=Fcc_ar,
                                                                                     f_g=Fg_ar,
                                                                                     f_g_sub=Fg_cc_ar,
                                                                                     calcG_params=calcG,
                                                                                     resistance_form=Resistance_flag)

# save ouputs in outdict 
LE = LE_C + LE_C_sub + LE_S
H = H_C + H_S + H_C_sub

Rn_C = L_nC + sn_ov
Rn_C_sub = Ln_C_sub + sn_un
Rn_S = Ln_S + sn_s
Rn = Rn_C + Rn_C_sub + Rn_S
Rn[Rn>1000] = np.nan
G_mod[G_mod>200] = np.nan
# visualizing outputs 
# save outputs to dictionary
model_outdict['LE_3SEB-3T'] = LE
model_outdict['LEc_3SEB-3T'] = LE_C
model_outdict['LEcc_3SEB-3T'] = LE_C_sub
model_outdict['H_3SEB-3T'] = H
model_outdict['Rn_3SEB-3T'] = Rn
model_outdict['G_3SEB-3T'] = G_mod
model_outdict['Tc_3SEB-3T'] = Tc_ar
model_outdict['Tcc_3SEB-3T'] = Tcc_ar
model_outdict['Ts_3SEB-3T'] = Ts_ar
model_outdict['Flags_3SEB-3T'] = flag_PT_all

# visualizing outputs 

fig, axes = plt.subplots(2,2, figsize=(10,8))
ax = axes[0,0]
im = ax.imshow(LE, vmin=0, vmax=600, cmap='PuBu', extent = te)
ax.set_title('LE')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('LE (W/$m^2$)')  # Add title to colorbar
le_mean = int(np.round(np.nanmean(LE),0))
ax.text(0.01,0.1, f'mean:\n{le_mean} W/$m^2$', transform=ax.transAxes)
ax = axes[0,1]
im = ax.imshow(H, vmin=0, vmax=600, cmap='OrRd',  extent = te)
ax.set_title('H')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('H (W/$m^2$)')  # Add title to colorbar
h_mean = int(np.round(np.nanmean(H),0))
ax.text(0.01,0.1, f'mean:\n{h_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,0]
im = ax.imshow(Rn, vmin=0, vmax=800, cmap='plasma',  extent = te)
ax.set_title('Rn')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Rn (W/$m^2$)')  # Add title to colorbar
rn_mean = int(np.round(np.nanmean(Rn),0))
ax.text(0.01,0.1, f'mean:\n{rn_mean} W/$m^2$', transform=ax.transAxes)

ax = axes[1,1]
im = ax.imshow(G_mod, vmin=0, vmax=200, cmap='copper',  extent = te)
ax.set_title('G')
# Add colorbar for only axes[0, 0]
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('G (W/$m^2$)')  # Add title to colorbar
g_mean = int(np.round(np.nanmean(G_mod),0))
ax.text(0.01,0.1, f'mean:\n{g_mean} W/$m^2$', transform=ax.transAxes)


plt.show()

# Model comparisons and evaluation
In this section we will compare the different model outputs. First, we will evaluated the model outputs against the EC tower measurements.

To make them as comparable as possible, we will estimate the 2D flux tower footprint to better characterize the pixels associated to the tower measurements.

The 2D EC tower flux footprint was estimated using the Flux Footprint Prediction (FFP) model as described in [Kljun et al. (2015)](https://doi.org/10.5194/gmd-8-3695-2015) and the code is available [here](https://footprint.kljun.net/)

This footprint estimates the weighted 2D area contributing to the EC sampling. As such, we can best compare the model results to the tower measurements by acquiring all pixels located within the estimated footprint and obtaining a weighted average for those pixels. The footprint was therefore rasterized and aligned to the arrays of the model output geometry to ease the comparison. 

See below for a visualization.

## Visualization of footprint area

In [ ]:
fp_dir = datadir / 'Footprints'
fp_file = fp_dir / f'surface_CC_202404161100_FPnorm_10cm.tif'
fp_fid = gdal.Open(str(fp_file))
fp_ar = fp_fid.GetRasterBand(1).ReadAsArray()
fp_ar[fp_ar < 0.0001] = np.nan

ulx, xres, xskew, uly, yskew, yres = fp_fid.GetGeoTransform()
lrx = ulx + (fp_fid.RasterXSize * xres)
lry = uly + (fp_fid.RasterYSize * yres)

te_10cm = [ulx, lrx, lry, uly]

# get 8m aligned footprint
fp_8m_file = fp_dir / f'surface_CC_202404161100_FPnorm_8m_aligned.tif'
fp_8m_fid = gdal.Open(str(fp_8m_file))
fp_8m_ar = fp_8m_fid.GetRasterBand(1).ReadAsArray()
fp_8m_ar[fp_8m_ar < 0.0001] = np.nan

fp_shp = fp_dir/'shp'/'surface_CC_202404161100_FPnorm.shp'
fp_gpd = gpd.read_file(fp_shp)

fig, axes = plt.subplots(2,2, figsize=(10,8))
ax1 = axes[0,0]
ax1.imshow(rgb_image, extent=te)
ax1.imshow(fp_ar, cmap='viridis', extent=te_10cm, alpha=0.7)
fp_gpd.plot(ax=ax1, facecolor='none', edgecolor='indianred', linewidth=1, alpha=0.75, label='footprint')
ax1.set_ylim(4156650, 4157050)
ax1.set_xlim(655150,655430)

ax2 = axes[0,1]

ax2.imshow(rgb_image, extent=te)
im1 = ax2.imshow(fp_ar, cmap='viridis', extent=te_10cm, alpha=0.7)
ax2.set_ylim(4156650,4156760)
ax2.set_xlim(655300,655430)
fp_gpd.plot(ax=ax2, facecolor='none', edgecolor='indianred', linewidth=1, alpha=0.75, label='footprint')
plt.colorbar(im1, ax=ax2, label='weighted flux contribution (-)',  shrink=0.65)

ax3 = axes[1,0]
ax3.imshow(model_outdict['LE_3SEB-PT'],vmin=0, vmax=600, cmap='PuBu', extent=te)
ax3.imshow(fp_8m_ar, cmap='viridis', extent=te, alpha=0.7)
ax3.set_ylim(4156650, 4157050)
ax3.set_xlim(655150,655430)
fp_gpd.plot(ax=ax3, facecolor='none', edgecolor='indianred', linewidth=1, alpha=0.75, label='footprint')

ax4 = axes[1,1]
ax4.imshow(model_outdict['LE_3SEB-PT'],vmin=0, vmax=600, cmap='PuBu', extent=te)
im2 = ax4.imshow(fp_8m_ar, cmap='viridis', extent=te, alpha=0.7)
ax4.set_ylim(4156650,4156760)
ax4.set_xlim(655300,655430)
fp_gpd.plot(ax=ax4, facecolor='none', edgecolor='indianred', linewidth=1, alpha=0.75, label='footprint')
plt.colorbar(im2, ax=ax4, label='weighted flux contribution (-)', shrink=0.65)
plt.show()

# Compare model results to measurements

## Get weighted model results over footprint 



In [ ]:
# observed data are saved in outdict

model_types = ['TSEB-PT', '3SEB-PT', '3SEB-2T','3SEB-3T']
fluxes = ['LE', 'H', 'Rn', 'G']

for model in model_types:
    for var in fluxes:
        var_ar = model_outdict[f'{var}_{model}']
        flags =  model_outdict[f'Flags_{model}']
        var_ar[var_ar<-100] = np.nan
        var_ar[var_ar>1000] = np.nan
        var_ar[flags > 5] = np.nan
        fp_8m_ar[fp_8m_ar==0] = np.nan
        # get weighted average
        avg_w = np.nansum(var_ar * fp_8m_ar)/np.nansum(fp_8m_ar)
        
        outdict[f'{var}_{model}'] = []
        outdict[f'{var}_{model}'].append(avg_w)

results_df = pd.DataFrame(outdict)
results_df

In [ ]:
# plot resutls 

import scipy.stats as st
    
model_types = ['TSEB-PT', '3SEB-PT', '3SEB-2T', '3SEB-3T']

h_obs = results_df['H_obs'].values
Rn_obs = results_df['Rn_obs'].values
G_obs = results_df['G_obs'].values
le_obs = results_df['LE_obs'].values
ec_res = Rn_obs - h_obs - le_obs - G_obs

le_res = Rn_obs - h_obs - G_obs
bowen_ratio = h_obs/le_obs
le_bowen = (Rn_obs - G_obs)/(1+bowen_ratio)
le_ens = (le_obs + le_bowen + le_res)/3
h_ens = Rn_obs - G_obs - le_ens

fig, axes = plt.subplots(2,2, figsize=(14,12))
i = 0
for model_type in model_types:
    #H
    h_mod = results_df[f'H_{model_type}'].values
    # Rn
    Rn_mod = results_df[f'Rn_{model_type}'].values
    # G
    G_mod = results_df[f'G_{model_type}'].values
    # LE
    le_mod = results_df[f'LE_{model_type}'].values

    if i == 0:
        ax = axes[0,0]
    elif i == 1:
        ax = axes[0,1]
    elif i == 2:
        ax = axes[1,0]
    else:
        ax = axes[1,1]


    #plt.scatter(le_mod[mask], le_obs[mask], color='dodgerblue', marker='o', label='LE (unclosed)', s=50)
    ax.scatter(le_mod, le_ens, color='dodgerblue', marker='s', label='LE', s=30)
    ax.scatter(h_mod, h_ens, color='indianred', marker='o', label='H', s=50)
    ax.scatter(Rn_mod, Rn_obs, color='orange', marker='d', label='Rn', s=50)
    ax.scatter(G_mod[G_obs > 0], G_obs[G_obs > 0], color='k', marker='^', label='G', s=50)
    ax.plot((0, 800), (0, 800), color='k', label='1:1 line', linestyle = '--')
    

    ax.legend()
    ax.set_xlim(0, 800)
    ax.set_ylim(0, 800)
    ax.set_xlabel(f'{model_type} modeled (W/$m^2$)', fontsize=14)
    if i == 0 or i == 2:
        ax.set_ylabel('EC Observed (W/$m^2$)', fontsize=14)
    #ax.set_title(f'{model_type}', fontsize=16)
    i = i + 1
plt.show()

# Comparison of modeled estimated component temperatures vs UAV contextual approach

Let us compare the component temperatures ($T_c$, $T_{cc}$ and $T_s$) retrieved from contextual methods to those estimated from TSEB-PT, 3SEB-PT and 3SEB-2T

In [ ]:
def model_metrics(X, Y, mask):
    rmse = np.sqrt(np.nanmean((X[mask] - Y[mask]) ** 2))
    cor = st.pearsonr(X[np.logical_and.reduce((mask,~np.isnan(X),~np.isnan(Y)))], Y[np.logical_and.reduce((mask,~np.isnan(Y),~np.isnan(X)))])[0]
    bias = np.nanmean(X[mask] - Y[mask])
    return rmse, cor, bias


# make 2 x 2 plot with different comparison
# get T_canopy
fig, axes = plt.subplots(2,2, figsize=(14,12))

# measured IRT
IRT_canopy = meteo_ds['T_CANOPY'].values[date_mask][0] + 273.15

# TSEB-PT estimated 
Tc_tseb = model_outdict['Tc_TSEB-PT']
Ts_tseb = model_outdict['Ts_TSEB-PT']

# 3SEB-PT estimated 
Tc_3seb_pt = model_outdict['Tc_3SEB-PT']
Tcc_3seb_pt = model_outdict['Tcc_3SEB-PT']
Ts_3seb_pt = model_outdict['Ts_3SEB-PT']

# 3SEB-2T estimated 
Tcc_3seb_2t = model_outdict['Tcc_3SEB-2T']
Ts_3seb_2t = model_outdict['Ts_3SEB-2T']

# error metrics for TSEB-PT
Tc_rmse, Tc_cor,_ = model_metrics(Tc_tseb,Tc_ar, np.logical_and(Tc_tseb>270, Tc_tseb<320))
Ts_rmse, Ts_cor,_ = model_metrics(Ts_tseb,Ts_ar, np.logical_and(Ts_tseb>270, Ts_tseb<320))

# TSEB-PT
ax = axes[0,0]
ax.scatter(Tc_tseb, Tc_ar, color='darkgreen', marker='o', label=f'$T_C$, r = {Tc_cor:.2f}, RMSE = {Tc_rmse:.1f}', s=50)
ax.scatter(Ts_tseb, Ts_ar, color='saddlebrown', marker='o', label=f'$T_S$, r = {Ts_cor:.2f}, RMSE = {Ts_rmse:.1f}', s=50)
ax.plot((0, 800), (0, 800), color='k', label='1:1 line', linestyle = '--')
ax.set_xlim(290,320)
ax.set_ylim(290,320)
ax.set_ylabel('Contextual UAV (K)', fontsize=16)
ax.set_xlabel('Modelled TSEB-PT (K)', fontsize=16)
ax.legend(fontsize=12)

# error metrics for 3SEB-PT
Tc_rmse, Tc_cor,_ = model_metrics(Tc_3seb_pt,Tc_ar, np.logical_and(Tc_3seb_pt>270, Tc_3seb_pt<320))
Tcc_rmse, Tcc_cor,_ = model_metrics(Tcc_3seb_pt,Tcc_ar, np.logical_and(Tcc_3seb_pt>270, Tcc_3seb_pt<320))
Ts_rmse, Ts_cor,_ = model_metrics(Ts_3seb_pt,Ts_ar, np.logical_and(Ts_3seb_pt>270, Ts_3seb_pt<320))

# 3SEB-PT
ax = axes[0,1]
ax.scatter(Tc_3seb_pt, Tc_ar, color='darkgreen', marker='o', label=f'$T_C$, r = {Tc_cor:.2f}, RMSE = {Tc_rmse:.1f}', s=50)
ax.scatter(Tcc_3seb_pt, Tcc_ar, color='yellowgreen', marker='o', label='$T_{C,sub}$'+ f', r = {Tcc_cor:.2f}, RMSE = {Tcc_rmse:.1f}', s=50)
ax.scatter(Ts_3seb_pt, Ts_ar, color='saddlebrown', marker='o', label=f'$T_S$, r = {Ts_cor:.2f}, RMSE = {Ts_rmse:.1f}', s=50)
ax.plot((0, 800), (0, 800), color='k', label='1:1 line', linestyle = '--')

ax.set_xlim(290,320)
ax.set_ylim(290,320)
ax.set_ylabel('Contextual UAV (K)', fontsize=16)
ax.set_xlabel('Modelled 3SEB-PT (K)', fontsize=16)
ax.legend(fontsize=12)

# 3SEB-2T
ax = axes[1,0]
# error metrics for 3SEB-2T
Tcc_rmse, Tcc_cor,_ = model_metrics(Tcc_3seb_2t,Tcc_ar, np.logical_and(Tcc_3seb_2t>270, Tcc_3seb_2t<320))
Ts_rmse, Ts_cor,_ = model_metrics(Ts_3seb_2t,Ts_ar, np.logical_and(Ts_3seb_2t>270, Ts_3seb_2t<320))

ax.scatter(Tcc_3seb_2t, Tcc_ar, color='yellowgreen', marker='o', label='$T_{C,sub}$'+ f', r = {Tcc_cor:.2f}, RMSE = {Tcc_rmse:.1f}', s=50)
ax.scatter(Ts_3seb_2t, Ts_ar, color='saddlebrown', marker='o', label=f'$T_S$, r = {Ts_cor:.2f}, RMSE = {Ts_rmse:.1f}', s=50)
ax.plot((270, 400), (270, 400), color='k', label='1:1 line', linestyle = '--')

ax.set_xlim(290,320)
ax.set_ylim(290,320)
ax.set_ylabel('Contextual UAV (K)', fontsize=16)
ax.set_xlabel('Modelled 3SEB-2T (K)', fontsize=16)
ax.legend(fontsize=12)


# get averages over footprint
Tc_tseb_fp = np.nansum(Tc_tseb * fp_8m_ar)/np.nansum(fp_8m_ar)
Tc_3seb_fp = np.nansum(Tc_3seb_pt * fp_8m_ar)/np.nansum(fp_8m_ar)
Tc_ar_fp = np.nansum(Tc_ar * fp_8m_ar)/np.nansum(fp_8m_ar)

ax = axes[1,1]
ax.scatter(Tc_tseb_fp, IRT_canopy, color='orange', marker='d', label=f'TSEB-PT', s=50)
ax.scatter(Tc_3seb_fp, IRT_canopy, color='indianred', marker='o', label=f'3SEB-PT', s=50)
ax.scatter(Tc_ar_fp, IRT_canopy, color='dodgerblue', marker='^', label=f'Contextual UAV', s=50)
ax.set_ylabel('Measured $T_c$ IRT (K)', fontsize=16)
ax.set_xlabel('Estimated $T_c$ (K)', fontsize=16)

ax.set_xlim(290,310)
ax.set_ylim(290,310)

ax.plot((0, 800), (0, 800), color='k', label='1:1 line', linestyle = '--')
ax.legend(fontsize=12)

plt.show()

:::{note}
There is some evidence that the contextual retrievals of temperature are overestimated according to the $T_C$ comparison with IRT measurements but also from the overestimation of H by 3SEB-3T. How do you think we could improve this or further evaluate this?
:::

:::{tip}
# **Next steps:** How does the partitioning of LE/ET change with each model variant? Compare the $LE_C$, $LE_{C,sub}$ and T/ET between model outputs. 
:::